<a href="https://colab.research.google.com/github/Crimsonking96/Complete-Python-3-Bootcamp/blob/master/MarketJudge_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The goal of this pipeline is to predict the next days movements (which sectors are hot and such)

- This block (successor to statement) tells us how current markets are positioned
- This information will then be fed into a system that will recognise positions of certain companys (their place in the market) and will convert the information (given my conditions) into a weight
- That weight will then be used in the Neural network of a SEMI-SUPERVISED ALGORITHM (The Judge), along with weightings of current Economy (interest rates, inflation, etc) it will then follow a Markovian Decision process (potentially a MCMC)
    - A dataset containg all the values of NYSE will be used to train the Judge
    - We'll also have scrapers using real time News to be used to Predict tomorrows outcomes
    - The Judge will be trained on a range of mid-cap-sized stocks from the afforementioned pool
    - It can identify and sort the stocks with the most growth (before as well as) following the pandemic, and those that are deemed volatile (determined using either the black scholes equation, or the Bachelier equation)
    - A Monte Carlo model will then be used to estimate a REASONABLE pot on Vulnerabilities ("Not just how to grow, but when"/ Should it Short, if so what ratio (warrant to stock)? Long call?)
- The Judge will then purchase these stocks on a simulation (investopedia)
    - Once Purchased the Machine will hold the stock until certain requirements have been broken, such as the stock dropping a significant percentage (1.33 points from what is deemed a peak). IT MUST SELL WHILE THE STOCKS ARE HOT!
- Once the simulation has been proven, I'll need to devise a way for the Judge to inreact with my portfolio

In RECAP, the Judge is a day trading model that will use my specific rules to buy/short and sell "Hot Stocks" and their respected Warrants (Mid Cap stocks with "potential" to grow or fall) based off of predictions of previouse market sector movements and current events. Although it uses data based off of the NYSE, it's ablility to identify which sectors will be profitable will allow it to identify Global stock prices.

In [ ]:
import os
import re
import json
import csv

import datetime
import random

import numpy as np
import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import matplotlib.dates as mdates


from scipy import *

import sklearn
from sklearn import *
from sklearn.preprocessing import StandardScaler as stscale
from sklearn.preprocessing import normalize as normal

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from urllib.request import urlopen
from bs4 import BeautifulSoup

from textblob import TextBlob

import sqlite3

#Scikit
from sklearn import *
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#TensorFlow
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cityblock, euclidean, cosine
import nltk, re, random
from nltk.tokenize import word_tokenize
from preprocessing import *

#Pytorch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from sklearn.datasets import make_classification
from torch import nn
from skorch import NeuralNetClassifier

import pytensor
from pytensor import tensor as pt

### Part 1

In [ ]:
def extract_sector_data(url):
    options = webdriver.EdgeOptions()
    driver = webdriver.Edge(options=options)
    driver.get(url)

    # Wait for the table to be present and visible
    wait = WebDriverWait(driver, 10)  # Adjust timeout as needed
    table = wait.until(EC.visibility_of_element_located((By.TAG_NAME, "table")))

    # Extract data from table rows
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    for row in rows[1:]:  # Skip the header row
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        data.append(row_data)

    driver.quit()
    return data

# Example usage
url = "https://www.sectorspdrs.com/allsectors"
sector_data = extract_sector_data(url)

# Process and print the extracted data (example)
for row in sector_data:
#     print(word_tokenize(cell) for cell in row])
#     for cell in row:
#         word_tokenize(cell)
#     print('\n' f'{row[1]} ({row[0]}), {row[4]} volume:({row[5]})')
    print(f'{row[0]}, {row[2]}, {row[3]} volume:({row[5]})')


In [ ]:
#Make this autonomous ^


sectors = pd.DataFrame([
    ['Symbol', 'Volume', 'Price'],
    ['XLC', '3.6M', 83.26],
    ['XLY', '3.42M', 175.97],
    ['XLP', '11.87M', 77.35],
    ['XLE', '16.25M', 93.22],
    ['XLF', '38.66M', 41.67],
    ['XLV', '7.95M', 143.70],
    ['XLI', '9.7M', 123.51],
    ['XLB', '4.1M', 91.54],
    ['XLRE', '7.95M',  38.04],
    ['XLK', '6.04M', 210.16],
    ['XLU', '12.39M', 72.73]
])

sectors

In [ ]:
equities = pd.read_csv('NYSE_Catalog_3.csv')

# Convert to numeric, removing dollar signs and commas
equities['mkt_cap'] = equities['mkt_cap'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['mkt_cap'] = pd.to_numeric(equities['mkt_cap'], errors='coerce')

equities['p/e'] = equities['p/e'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['p/e'] = pd.to_numeric(equities['p/e'], errors='coerce')

equities['avg_volume'] = equities['avg_volume'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['avg_volume'] = pd.to_numeric(equities['avg_volume'], errors='coerce')

equities['previous_close'] = equities['previous_close'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['previous_close'] = pd.to_numeric(equities['previous_close'], errors='coerce')

equities['open'] = equities['open'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['open'] = pd.to_numeric(equities['open'], errors='coerce')

equities['high'] = equities['high'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['high'] = pd.to_numeric(equities['high'], errors='coerce')

equities['low'] = equities['low'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['low'] = pd.to_numeric(equities['low'], errors='coerce')

equities['52_wkH'] = equities['52_wkH'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['52_wkH'] = pd.to_numeric(equities['52_wkH'], errors='coerce')

equities['52_wkL'] = equities['52_wkL'].astype(str).str.replace(r'[$,]', '', regex=True)
equities['52_wkL'] = pd.to_numeric(equities['52_wkL'], errors='coerce')

# Handling missing or invalid values (if necessary)
equities.fillna(0, inplace=True)
equities.replace([np.inf, -np.inf], np.nan, inplace=True)

# Filtering the DataFrame
mid_cap_stocks = equities[(equities['mkt_cap'] >= 10**9) & (equities['mkt_cap'] <= (10**9)*30)]

mid_cap_stocks.to_csv('mid_cap_stocks.csv', index=False)

In [ ]:
mid_cap_stocks.head()

In [ ]:
# 1. Style and Theme
sns.set_theme(style="whitegrid")  # Use a clean, whitegrid style
plt.figure(figsize=(12, 8))        # Adjust figure size as needed

# 2. Customized Color Palette
num_industries = equities['industry'].nunique()
colors = sns.color_palette("husl", n_colors=num_industries)  # Use "husl" for vibrant, distinct colors

# 3. Scatter Plot with Refined Aesthetics
sns.scatterplot(
    data=equities, x='mkt_cap', y='avg_volume', hue='industry',
    size='mkt_cap', sizes=(50, 800), alpha=0.7,  # Increase sizes, add transparency for overlap
    palette=colors, edgecolor='black', linewidth=0.5  # Add black edges to points
)

# 4. Enhanced Axis Labels and Title
plt.xlabel('Market Capitalization (in Billions USD)', fontsize=12)
plt.ylabel('Average Daily Volume (in Millions)', fontsize=12)
plt.title('Market Capitalization vs. Average Volume by Industry (Mid-Cap Stocks)', fontsize=14)

# 5. Logarithmic X-Axis for Better Visualization (if data is skewed)
plt.xscale('log')  # Use logarithmic scale if market caps vary widely

# 6. Refined Legend Placement
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Industry', fontsize=10)
for text in legend.get_texts():
    text.set_fontsize(10)  # Reduce legend font size for better aesthetics

# 7. Grid Adjustments
sns.despine(left=True, bottom=True)  # Remove top and right spines for cleaner look
plt.grid(axis='y', linestyle='--')    # Show horizontal gridlines only

# 8. Show the Plot
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


### scrape newsites, none in particular, We'll need web crawlers seeking specified info.

In [ ]:
def get_stock_price(ticker, exchange):
    """Retrieves the current stock price for the given ticker and exchange from Google Finance.

    Also handles printing the results (success or error).
    """

    url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')

    # Target the specific element containing the price
    price_div = bs.find('div', class_='YMlKec fxKbKc')
    if price_div:  # Check if price element is found
        price_text = price_div.get_text()

        # Convert to float (or handle cases where price isn't available)
        try:
            price = float(price_text.replace('$', ''))
            print(f"The current price of {ticker} is ${price:.2f}")
        except ValueError:
            print(f"Price information for {ticker} is not available at this time.")
    else:
        print(f"Price information for {ticker} is not available at this time.")  # Handle missing price element

get_stock_price('GCEHF', 'OTCMKTS')


In [ ]:
# Take newsource, place into df
np.ndarray([
    Title,
    Traffick_count, #how many people have clciked the article; rows should be sorted largest to smallest
    Word_count,
    Date,
    Url
    ])

#we'de like to be able to determine whether the article has a positive connotation or a negative one
# ex: if word positive

    # positive/word_count

# else: negative/word_count

#allowing us to comprehend what impact the article will have on the reader as well as some consensus as to what action they may succeed

money_news = [
    {'google_finance': 'Google Finance', 'url': 'https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pWVXlnQVAB?hl=en-US&gl', 'category': 'finance'},
    {'yahoo_finance': 'Yahoo Finance', 'url': 'https://finance.yahoo.com/topic/stock-market-news/', 'category': 'finance'},
    {'msn_finance': 'MSN Money', 'url': 'https://www.msn.com/en-us/money?id=a6qja2', 'category': 'finance'},
    {'investopedia': 'Investopedia', 'url': 'https://www.investopedia.com/news-4427706', 'category': 'finance'},
    {'pew_research': 'Pew Research Center', 'url': 'https://www.pewresearch.org/topic/economy-work/', 'category': 'research'}
]

In [ ]:
# Data storage (using pandas DataFrame for now)
data = pd.DataFrame(columns=["source", "url", "title", "summary", "tickers", "sentiment"])

money_news = [
    {'google_finance': 'Google Finance', 'url': 'https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pWVXlnQVAB?hl=en-US&gl', 'category': 'finance'},
    {'yahoo_finance': 'Yahoo Finance', 'url': 'https://finance.yahoo.com/topic/stock-market-news/', 'category': 'finance'},
    {'msn_finance': 'MSN Money', 'url': 'https://www.msn.com/en-us/money?id=a6qja2', 'category': 'finance'},
    {'investopedia': 'Investopedia', 'url': 'https://www.investopedia.com/news-4427706', 'category': 'finance'},
    {'pew_research': 'Pew Research Center', 'url': 'https://www.pewresearch.org/topic/economy-work/', 'category': 'research'}
]

def crawl_page(source, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract headlines and summaries (adapt to each website's structure)
    headlines = soup.find_all('a', class_='DY5T1d RZIKme')  # Google Finance example
    summaries = soup.find_all('div', class_='SbNwzf')      # Google Finance example

    for headline, summary in zip(headlines, summaries):
        title = headline.text
        summary_text = summary.text

        # Extract stock tickers
        tickers = re.findall(r'\b[A-Z]{1,6}\b', title + " " + summary_text)
        # Basic ticker pattern (uppercase, 1-6 letters), refine as needed

        # Sentiment analysis
        sentiment = TextBlob(summary_text).sentiment.polarity
        if sentiment > 0:
            sentiment = "positive"
        elif sentiment < 0:
            sentiment = "negative"
        else:
            sentiment = "neutral"

        data.loc[len(data)] = [source, url, title, summary_text, tickers, sentiment]


# Crawl through each source
for source_dict in money_news:
    source_name = list(source_dict.keys())[1]
    source_url = source_dict[source_name]
    crawl_page(source_name, source_url)

print(data)


In [ ]:
# Data storage
data = pd.DataFrame(columns=["source", "url", "title", "traffic_count", "word_count", "date", "tickers", "sentiment"])

money_news = [
    {'google_finance': 'Google Finance', 'url': 'https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pWVXlnQVAB?hl=en-US&gl', 'category': 'finance'},
    {'yahoo_finance': 'Yahoo Finance', 'url': 'https://finance.yahoo.com/topic/stock-market-news/', 'category': 'finance'},
    {'msn_finance': 'MSN Money', 'url': 'https://www.msn.com/en-us/money?id=a6qja2', 'category': 'finance'},
    {'investopedia': 'Investopedia', 'url': 'https://www.investopedia.com/news-4427706', 'category': 'finance'},
    {'pew_research': 'Pew Research Center', 'url': 'https://www.pewresearch.org/topic/economy-work/', 'category': 'research'}
]

def crawl_page(source, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Example selectors for Google Finance (adapt as needed)
    articles = soup.find_all('article')

    for article in articles:
        try:
            title_elem = article.find('a', class_='DY5T1d RZIKme')
            title = title_elem.text.strip()

            # Traffic count (replace with actual logic if available)
            traffic_count = 0  # Placeholder, needs implementation

            url = title_elem['href']
            # Word count
            summary_elem = article.find('div', class_='SbNwzf')
            summary_text = summary_elem.text.strip()
            word_count = len(summary_text.split())

            # Date (replace with actual logic if available)
            date = dt.datetime.today()  # Placeholder, needs implementation

            # Tickers
            tickers = re.findall(r'\b[A-Z]{1,6}\b', title + " " + summary_text)

            # Sentiment
            sentiment = TextBlob(summary_text).sentiment.polarity
            sentiment_label = np.select([sentiment > 0, sentiment < 0], ["positive", "negative"], "neutral")

            data.loc[len(data)] = [source, url, title, traffic_count, word_count, date, tickers, sentiment_label]

        except (AttributeError, TypeError):
            pass  # Handle cases where elements are not found

# Crawl and sort by traffic count
for source_dict in money_news:
    source_name = list(source_dict.keys())[1]
    source_url = source_dict[source_name]
    print(crawl_page(source_name, source_url))

# Sort by traffic count (descending)
data.sort_values(by="traffic_count", ascending=False, inplace=True)

print(data.head())


In [ ]:
# additionally crawlers for Twitter may allow us to confirm what action the pre-established population may take
# either by searching for terms relevent to the news source, or spotting additional clues as to what market actions are hot
np.ndarray([
    Topic, #what data was identified by the crawler
    Traffick, #of the posts scraped, what was the percent of specific topics present
    Hot, #must have a percent of over %75 to be considered for the training set
    Date, #must be today
])

In [ ]:
cpi_discrete =  pd.read_csv("CPILFESL.csv") #consumer discretionary
stky_shft = pd.read_csv("CORESTICKM159SFRBATL.csv") #Consumer discretionary sticky cpi
mrtge_30y = pd.read_csv("MORTGAGE30US.csv") #30yr mortgage rate
prime_loan = pd.read_csv("MPRIME.csv") #top 25 banks short term loans
ffr = pd.read_csv("DFF.csv") #Federal funds rate
unemplyment = pd.read_csv("UNRATE.csv") #rate of Unemployment
mdn_cpi = pd.read_csv("MEDCPIM158SFRBCLE.csv") #Median CPI
cpi_cty_avg = pd.read_csv("CPIAUCSL.csv")

### take the previously established data and convert them to weights for our ANN